In [ ]:
# dataset: https://www.kaggle.com/uciml/pima-indians-diabetes-database
# github: https://github.com/pranav567/AI-ML-Lab/tree/master/experiment-4

In [283]:
import pandas as pd
import numpy as np

diabetes = pd.read_csv('diabetes.csv')
diabetes = diabetes.dropna()
print(diabetes)

     Glucose  BloodPressure  SkinThickness  Insulin   BMI  Age  Outcome
0        148             72             35        0  33.6   50        1
1         85             66             29        0  26.6   31        0
2        183             64              0        0  23.3   32        1
3         89             66             23       94  28.1   21        0
4        137             40             35      168  43.1   33        1
..       ...            ...            ...      ...   ...  ...      ...
763      101             76             48      180  32.9   63        0
764      122             70             27        0  36.8   27        0
765      121             72             23      112  26.2   30        0
766      126             60              0        0  30.1   47        1
767       93             70             31        0  30.4   23        0

[768 rows x 7 columns]


In [284]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(diabetes,diabetes['Outcome'],test_size=0.2)


In [285]:
df = x_train.copy()
diabetes_yes = df.where(df['Outcome']==1).dropna()
diabetes_yes_prob = len(diabetes_yes)/len(df)
diabetes_no = df.where(df['Outcome']==0).dropna()
diabetes_no_prob = len(diabetes_no)/len(df)

print(diabetes_yes_prob,diabetes_no_prob)

0.34039087947882735 0.6596091205211726


In [286]:
columns = df.iloc[:,:6].columns.to_list()
classes = 15
column_details = []
for column_name in columns:
    min_val = min(df[column_name].to_list())
    max_val = max(df[column_name].to_list())
    class_size = round((max_val-min_val)/classes , 2)

    low_bounds = list(np.arange(min_val,max_val,class_size))
    for i in range(len(low_bounds)):
        low_bounds[i] = round(low_bounds[i],2)
    x = low_bounds[-1]
    low_bounds.append(round(x + class_size,2))
    # print(low_bounds)
    # mid_class = round((class_size/2) , 2)
    dict = {}
    # start_mid = mid_class
    # df_temp = df[column_name]
    # print(df_temp)
    # print(len(diabetes_yes) , len(diabetes_no))

    for i in range(classes):
#         # print(start_mid)
        low_bound = low_bounds[i]
        upp_bound = low_bounds[i+1]

        yes_count = diabetes_yes.where(diabetes_yes[column_name]>=low_bound).dropna()
        if i<classes-1:
            yes_count = yes_count.where(yes_count[column_name]<upp_bound).dropna()
        else:
            yes_count = yes_count.where(yes_count[column_name]<=upp_bound).dropna()

        no_count = diabetes_no.where(diabetes_no[column_name]>=low_bound).dropna()
        if i<classes-1:
            no_count = no_count.where(no_count[column_name]<upp_bound).dropna()
        else:
            no_count = no_count.where(no_count[column_name]<=upp_bound).dropna()


#         # print(len(yes_count)/len(diabetes_yes))
#         # print(len(yes_count)/len(diabetes_yes) , len(no_count)/len(diabetes_no))

        dict[low_bounds[i]] = [len(yes_count)/len(diabetes_yes) , len(no_count)/len(diabetes_no)]
#         # print([low_bound,upp_bound])
#         start = round((start_mid + class_size),2)

    column_details.append([column_name, dict , class_size])
print(column_details)

[['Glucose', {0.0: [0.004784688995215311, 0.0049382716049382715], 13.27: [0.0, 0.0], 26.54: [0.0, 0.0], 39.81: [0.0, 0.0], 53.08: [0.0, 0.012345679012345678], 66.35: [0.004784688995215311, 0.06666666666666667], 79.62: [0.03349282296650718, 0.1580246913580247], 92.89: [0.09569377990430622, 0.24444444444444444], 106.16: [0.11961722488038277, 0.18271604938271604], 119.43: [0.16267942583732056, 0.145679012345679], 132.7: [0.12440191387559808, 0.09135802469135802], 145.97: [0.12440191387559808, 0.05185185185185185], 159.24: [0.12440191387559808, 0.01728395061728395], 172.51: [0.11483253588516747, 0.012345679012345678], 185.78: [0.09090909090909091, 0.012345679012345678]}, 13.27], ['BloodPressure', {0.0: [0.04784688995215311, 0.0345679012345679], 7.6: [0.0, 0.0], 15.2: [0.0, 0.0], 22.8: [0.004784688995215311, 0.0], 30.4: [0.0, 0.0], 38.0: [0.004784688995215311, 0.012345679012345678], 45.6: [0.03827751196172249, 0.04938271604938271], 53.2: [0.04784688995215311, 0.1506172839506173], 60.8: [0.1

In [287]:
def get_outcome(input_data):
    probabilities = []

    for i in range(6):
        value = input_data[i]
        prob_dict = column_details[i][1]
        class_range = column_details[i][2]

        key_list = list(prob_dict.keys())
        
        for key in key_list:
            low = key
            up = key+class_range

            if key_list[-1] == key:
                if value>=low and value <= up:
                    probabilities.append(prob_dict[key])
                    break
            else:
                if value>=low and value < up:
                    probabilities.append(prob_dict[key])
                    break

    yes=1
    no=1
    for i in probabilities:
        if i[0] == 0 and i[1]==0:
            continue
        yes=yes*i[0]
        no=no*i[1]
    
    if yes==1 or no==1:
        return 0

    product = [yes*diabetes_yes_prob , no*diabetes_no_prob]
    total = yes*diabetes_yes_prob + no*diabetes_no_prob

    
    outcome = [product[0]/total, product[1]/total]

    if outcome[0] > outcome[1]:
        return 1
    else:
        return 0

In [288]:
test = x_test.iloc[:,:6].copy()
test_set = []
for i in range(154):
    test_set.append(test.iloc[i].to_list())

by_nb = []
for i in test_set:
    by_nb.append(get_outcome(i))


In [289]:
by_dataset = y_test.to_list()
correct = 0
for i in range(len(by_nb)):
    if by_nb[i]==by_dataset[i]:
        correct = correct + 1

print(correct/len(by_nb)*100)

79.87012987012987


In [290]:
x_test

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Age,Outcome
511,139,62,17,210,22.1,21,0
478,126,74,38,75,25.9,39,0
13,189,60,23,846,30.1,59,1
9,125,96,0,0,0.0,54,1
23,119,80,35,0,29.0,29,1
...,...,...,...,...,...,...,...
588,176,86,27,156,33.3,52,1
529,111,65,0,0,24.6,31,0
369,133,102,28,140,32.8,45,1
437,147,75,0,0,29.9,28,0


In [291]:
print("Enter values for the following columns:\n" + ", ".join(columns) + "\n")
user_input  = input()
user_input = user_input.split(' ')
input_data = [float(i) for i in user_input]
print(input_data)
get_outcome(input_data)

Enter values for the following columns:
Glucose, BloodPressure, SkinThickness, Insulin, BMI, Age

[111.0, 65.0, 0.0, 0.0, 24.6, 31.0]


0

In [292]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

clf = GaussianNB()

x_tr, x_te, y_tr, y_te = train_test_split(diabetes.iloc[:,:6],diabetes['Outcome'],test_size=0.2)

clf.fit(x_tr,y_tr)
result = clf.predict(x_te)
accuracy_score(y_te, result) *100

77.92207792207793